In [59]:
#import packages
import scanpy as sc
import pandas as pd
import squidpy as sq
import numpy as np
from scipy.spatial import *
from sklearn.preprocessing import *

from sklearn.metrics import *
from scipy.spatial.distance import *

In [ ]:
import SDMBench.SDMBench as sdmbench
sdmbench=sdmbench.sdmbench()

In [ ]:
#First,you must download the data. We test one of 10x visium datasets from LIBD human dorsolateral prefrontal cortex, and you can download processed h5ad file in the /SDMBench/Data dir.
#Then read 151507.h5ad

In [63]:
adata=sc.read_h5ad('/SDMBench/Data/151507.h5ad')
adata_valid = adata[np.logical_not(adata.obs['Region'].isna())]#remove NAN
adata_valid

View of AnnData object with n_obs × n_vars = 4221 × 33538
    obs: 'in_tissue', 'array_row', 'array_col', 'Region', 'pred_1', 'pred_2', 'pred_3', 'pred_4', 'pred_5', 'pred_6', 'pred_7', 'pred_8', 'pred_9', 'pred_10', 'pred_refined_1', 'pred_refined_2', 'pred_refined_3', 'pred_refined_4', 'pred_refined_5', 'pred_refined_6', 'pred_refined_7', 'pred_refined_8', 'pred_refined_9', 'pred_refined_10'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'ari', 'memory', 'refined_ari', 'spatial', 'time'
    obsm: 'spatial'

In [67]:
#read result of your method(txt file)
pred=pd.read_csv('new_method.txt')

In [68]:
#compute ari
ari=sdmbench.compute_ARI(adata_valid,f'Region',f'pred')
ari

0.42130812976368504

In [69]:
#compute nmi
nmi=sdmbench.compute_NMI(adata_valid,f'Region',f'pred')
nmi

0.550818750430498

In [70]:
#compute CHAOS
chaos=sdmbench.compute_CHAOS(adata_valid,f'pred')
chaos

0.06066783744035732

In [71]:
#compute PAS
pas=sdmbench.compute_PAS(adata_valid,f'pred',spatial_key='spatial')
pas

0.17720919213456526

In [72]:
#compute ASW
asw=sdmbench.compute_ASW(adata_valid,f'pred',spatial_key='spatial')
asw

-0.029425595396403586

In [73]:
#compute HOM
hom=sdmbench.compute_HOM(adata_valid,f'Region',f'pred')
hom

0.5419495519799414

In [74]:
#compute COM
com=sdmbench.compute_COM(adata_valid,f'Region',f'pred')
com

0.5599830739241538

In [75]:
#compute marker_score
moranI,gearyC=sdmbench.marker_score(adata_valid,f'Region')
moranI,gearyC

/home/workspace2/zhaofangyuan/anaconda3/envs/zhuanhuan/lib/python3.8/site-packages/scanpy/preprocessing/_simple.py:524: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[key_n_counts] = counts_per_cell


  0%|          | 0/100 [00:00<?, ?/s]

  0%|          | 0/100 [00:00<?, ?/s]

0.277929965839376 0.7225470532588847


In [76]:
output_df = pd.DataFrame([[nmi,hom,com,chaos,pas,asw,moranI,gearyC]],
               index = [pred.columns[0]],#your method name
               columns=[['Accuracy','Accuracy','Accuracy','Continuity','Continuity','Continuity','Marker score','Marker score'],
                        ['NMI','HOM','COM','CHAOS','PAS','ASW','Moran\'I','Geary\'s C']])
output_df.to_excel('./output_result.xlsx')